# Visualisatie riool data
## Data importeren

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import geopandas as gpd
from datetime import date,datetime,timedelta
from matplotlib.ticker import ScalarFormatter
import numpy as np
from matplotlib.colors import LinearSegmentedColormap, to_rgba_array

In [ ]:

sewage_rows = []
for vr in range(1,26):
    sewage_import = pd.read_json("https://coronadashboard.rijksoverheid.nl/json/VR{:02d}.json".format(vr))
    for row in sewage_import['sewer']['values']:
        sewage_rows.append({
            'VR': 'VR{:02d}'.format(vr),
            'Date': pd.to_datetime(row['date_unix'], unit="s"),
            'VirusParts': row['average']
        })

sewer = pd.DataFrame(sewage_rows)
dateOfStats = sewer.Date.max()
sewer

In [ ]:
vrNames = pd.read_csv('https://raw.githubusercontent.com/mzelst/covid-19/master/misc/nursery-homes-locations.csv').drop(columns=['Aantal_locaties'])
sewer = sewer.join(vrNames.set_index('Security_region_code'), on='VR')
sewer

In [ ]:
map_data = gpd.read_file("https://cartomap.github.io/nl/wgs84/veiligheidsregio_2021.geojson")
map_data

## Data voorbereiden

In [ ]:
sewer_growth = sewer.copy().groupby(by=['VR', 'Security_region_name']).rolling(on="Date", window=7).mean()
sewer_growth['VirusGrowth_7d'] = sewer_growth.VirusParts.pct_change().replace([np.inf, -np.inf], 0).fillna(0).rolling(window=7).mean() * 100
sewer_growth = sewer_growth.reset_index()
sewer_growth

#sum7d_totals = ageGroupStats.copy().groupby(by=["GGD_name", "Age_group"]).rolling(on="Date", window=7).sum().reset_index()
#sum7d_totals["growth"] = sum7d_totals.Positive_cases.rolling(window=7).sum().pct_change().replace([np.inf, -np.inf], 0).fillna(0).rolling(window=7).mean() * 100
#sum7d_totals


In [ ]:
fallers = sewer_growth[(sewer_growth.Date == dateOfStats) & (sewer_growth.VirusGrowth_7d < -5)].VR.to_list()
risers = sewer_growth[(sewer_growth.Date == dateOfStats) & (sewer_growth.VirusGrowth_7d >= 0)].VR.to_list()

In [ ]:
mins = sewer_growth.groupby(by=["Date"]).min()
maxs = sewer_growth.groupby(by=["Date"]).max()
medians = sewer_growth.groupby(by=["Date"]).median()
avg = sewer_growth.groupby(by=["Date"]).mean()
avg["min"] = mins.VirusGrowth_7d
avg["max"] = maxs.VirusGrowth_7d
avg["median"] = medians.VirusGrowth_7d
avg = avg.reset_index()
avg

## Plots configureren

In [ ]:
sns.set_style("whitegrid")
sns.set_context("talk", font_scale=1, rc={"lines.linewidth": 3, "font.size":10,"axes.titlesize":24,"axes.labelsize":18})
sns.set_palette(sns.color_palette(['#CC6677', '#332288', '#DDCC77', '#117733', '#88CCEE', \
                    '#882255', '#44AA99', '#999933', '#AA4499', '#DDDDDD', \
                    '#000000']))

def tuftefy(ax):
    """Remove spines and tick position markers to reduce ink."""
    # 
    ax.spines["top"].set_visible(False)
    ax.spines["right"].set_visible(False)
    ax.spines["left"].set_visible(False)
    ax.spines["bottom"].set_visible(True)
    ax.spines["bottom"].set_color('grey')

    ax.grid(color="w", alpha=0.5)
    ax.get_yaxis().grid(True)
    ax.get_xaxis().grid(False)

def fancy_titles(t1, t2, ax=None):
  if ax:
    ax.set_title(t1, loc='left', fontsize=18)
    ax.set_title(t2, loc='right', fontsize=13, color='grey')
  else:
    plt.title(t1, loc='left', fontsize=18)
    plt.title(t2, loc='right', fontsize=13, color='grey')

def __BuRd():
    """
    Define colormap 'BuRd'.
    """
    clrs = ['#2166AC', '#4393C3', '#92C5DE', '#D1E5F0', '#F7F7F7',
            '#FDDBC7', '#F4A582', '#D6604D', '#B2182B']
    return LinearSegmentedColormap.from_list('BuRd', clrs)

## Plots

### Trends

In [ ]:
fig = plt.figure()
fig.set_size_inches(16, 10)

dt = sewer_growth[sewer_growth.Date > '2021-11-30']
g = sns.lineplot(data=dt, x='Date', y='VirusGrowth_7d', hue='VR')
plt.axhline(y=0.0, ls='-', c='.8', zorder=1)
tuftefy(g)
g.xaxis.set_major_formatter(mdates.ConciseDateFormatter(mdates.AutoDateLocator()))
g.set_title("Riooldeeltjes groei per Veiligheids regio", fontdict={'fontsize':14,'fontweight': 'bold'})
g.set(xlabel="", ylabel="Groei (%)")
g.text(dt.Date.min(), dt.VirusGrowth_7d.min(), "bron: corona dashboard update " + dateOfStats.strftime('%Y-%m-%d'), ha="left", fontsize=10, color='grey')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0)

In [ ]:
fig = plt.figure()
fig.set_size_inches(16, 10)

dt = sewer_growth[sewer_growth.Date > '2021-11-30']
g = sns.scatterplot(data=dt, x='Date', y='VirusGrowth_7d', hue='VR')
plt.axhline(y=0.0, ls='-', c='.8', zorder=1)
tuftefy(g)
g.xaxis.set_major_formatter(mdates.ConciseDateFormatter(mdates.AutoDateLocator()))
g.set_title("Riooldeeltjes groei per Veiligheids regio", fontdict={'fontsize':14,'fontweight': 'bold'})
g.set(xlabel="", ylabel="Groei (%)")
g.text(dt.Date.min(), dt.VirusGrowth_7d.min(), "bron: corona dashboard update " + dateOfStats.strftime('%Y-%m-%d'), ha="left", fontsize=10, color='grey')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0)

In [ ]:
fig = plt.figure()
fig.set_size_inches(8, 6)
dt = avg[avg.Date > '2021-12-31']
g = sns.lineplot(data=dt, x='Date', y='median')
plt.fill_between(data=dt, x='Date', y1='min', y2='max',zorder=1, alpha=0.2)
plt.axhline(y=0.0, ls='-', c='.8', zorder=1)
tuftefy(g)
g.xaxis.set_major_formatter(mdates.ConciseDateFormatter(mdates.AutoDateLocator()))
g.set_title("Riooldeeltjes groei per Veiligheids regio", fontdict={'fontsize':14,'fontweight': 'bold'})
g.set(xlabel="", ylabel="Groei (%)")
g.text(dt.Date.min(), dt['min'].min(), "bron: corona dashboard update " + dateOfStats.strftime('%Y-%m-%d'), ha="left", fontsize=10, color='grey')

In [ ]:
fig = plt.figure()
fig.set_size_inches(16, 10)

dt = sewer_growth[sewer_growth.Date > '2021-11-30'].copy()
dt['Categorie'] = 'Stabiel'
dt.loc[dt.VR.isin(fallers), 'Categorie'] = 'Dalers'
dt.loc[dt.VR.isin(risers), 'Categorie'] = 'Stijgers'
#dt['Categorie'] = np.where(dt.VR.isin(fallers), 'Dalers', 'Stijgers')
g = sns.lineplot(data=dt, x='Date', y='VirusGrowth_7d', hue='Categorie', ci=95, err_kws={ "edgecolor" : None })
plt.axhline(y=0.0, ls='-', c='.8', zorder=1)
tuftefy(g)
g.xaxis.set_major_formatter(mdates.ConciseDateFormatter(mdates.AutoDateLocator()))
g.set_title("Riooldeeltjes groei per Veiligheids regio", fontdict={'fontsize':14,'fontweight': 'bold'})
g.set(xlabel="", ylabel="Groei (%)")
g.text(dt.Date.min(), dt.VirusGrowth_7d.min() * 0.9, "bron: corona dashboard update " + dateOfStats.strftime('%Y-%m-%d'), ha="left", fontsize=10, color='grey')
#plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0)

In [ ]:

fig = plt.figure()
fig.set_size_inches(16, 10)

dt = sewer_growth[sewer_growth.Date > '2021-11-30'].copy()
dt['Categorie'] = 'Stijgers'
dt.loc[dt.VR.isin(fallers), 'Categorie'] = 'Dalers'
#dt['Categorie'] = np.where(dt.VR.isin(fallers), 'Dalers', 'Stijgers')
g = sns.lineplot(data=dt, x='Date', y='VirusGrowth_7d', hue='Categorie', ci=95, err_kws={ "edgecolor" : None })
plt.axhline(y=0.0, ls='-', c='.8', zorder=1)
tuftefy(g)
g.xaxis.set_major_formatter(mdates.ConciseDateFormatter(mdates.AutoDateLocator()))
g.set_title("Riooldeeltjes groei per Veiligheids regio", fontdict={'fontsize':14,'fontweight': 'bold'})
g.set(xlabel="", ylabel="Groei (%)")
g.text(dt.Date.min(), dt.VirusGrowth_7d.min() * 0.9, "bron: corona dashboard update " + dateOfStats.strftime('%Y-%m-%d'), ha="left", fontsize=10, color='grey')
#plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0)

In [ ]:

fig = plt.figure()
fig.set_size_inches(16, 10)

dt = sewer_growth[sewer_growth.Date > '2020-12-31'].copy()
dt['Categorie'] = 'Stijgers'
dt.loc[dt.VR.isin(fallers), 'Categorie'] = 'Dalers'
#dt['Categorie'] = np.where(dt.VR.isin(fallers), 'Dalers', 'Stijgers')
g = sns.lineplot(data=dt, x='Date', y='VirusParts', hue='Categorie', ci=95, err_kws={ "edgecolor" : None })
#plt.axhline(y=0.0, ls='-', c='.8', zorder=1)
tuftefy(g)
g.xaxis.set_major_formatter(mdates.ConciseDateFormatter(mdates.AutoDateLocator()))
g.set_title("Riooldeeltjes per Veiligheids regio", fontdict={'fontsize':14,'fontweight': 'bold'})
g.set(xlabel="", ylabel="Riooldeeltjes")
g.text(dt.Date.min(), dt.VirusGrowth_7d.min() * 0.9, "bron: corona dashboard update " + dateOfStats.strftime('%Y-%m-%d'), ha="left", fontsize=10, color='grey')
#plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0)

In [ ]:

fig = plt.figure()
fig.set_size_inches(16, 10)

dt = sewer_growth[sewer_growth.Date > '2020-12-31'].copy()
dt['Categorie'] = 'Stabiel'
dt.loc[dt.VR.isin(fallers), 'Categorie'] = 'Dalers'
dt.loc[dt.VR.isin(risers), 'Categorie'] = 'Stijgers'
#dt['Categorie'] = np.where(dt.VR.isin(fallers), 'Dalers', 'Stijgers')
g = sns.lineplot(data=dt, x='Date', y='VirusParts', hue='Categorie', ci=95, err_kws={ "edgecolor" : None })
#plt.axhline(y=0.0, ls='-', c='.8', zorder=1)
tuftefy(g)
g.xaxis.set_major_formatter(mdates.ConciseDateFormatter(mdates.AutoDateLocator()))
g.set_title("Riooldeeltjes per Veiligheids regio", fontdict={'fontsize':14,'fontweight': 'bold'})
g.set(xlabel="", ylabel="Riooldeeltjes")
g.text(dt.Date.min(), dt.VirusGrowth_7d.min() * 0.9, "bron: corona dashboard update " + dateOfStats.strftime('%Y-%m-%d'), ha="left", fontsize=10, color='grey')
#plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0)

In [ ]:

#fig = plt.figure()
#fig.set_size_inches(16, 10)

dt = sewer_growth[sewer_growth.Date > '2021-12-31'].copy()

g = sns.relplot(data=dt, kind='line', x='Date', y ='VirusGrowth_7d', col='Security_region_name', col_wrap=5)
g = g.map(plt.axhline, y=0.0, ls="-", c=".8", zorder=1)

for ax in g.axes:
  ax.xaxis.set_major_formatter(mdates.ConciseDateFormatter(mdates.AutoDateLocator()))
  ax.set(ylim=[-40, 40])
  tuftefy(ax)

g.set_titles(col_template = '{col_name}')
g.set_axis_labels("", "Groei percentage")
g.despine()
plt.tight_layout()

#dt['Categorie'] = 'Stabiel'
#dt.loc[dt.VR.isin(fallers), 'Categorie'] = 'Dalers'
#dt.loc[dt.VR.isin(risers), 'Categorie'] = 'Stijgers'
##dt['Categorie'] = np.where(dt.VR.isin(fallers), 'Dalers', 'Stijgers')
#g = sns.lineplot(data=dt, x='Date', y='VirusParts', hue='Categorie', ci=95, err_kws={ "edgecolor" : None })
##plt.axhline(y=0.0, ls='-', c='.8', zorder=1)
#tuftefy(g)
#g.xaxis.set_major_formatter(mdates.ConciseDateFormatter(mdates.AutoDateLocator()))
#g.set_title("Riooldeeltjes per Veiligheids regio", fontdict={'fontsize':14,'fontweight': 'bold'})
#g.set(xlabel="", ylabel="Riooldeeltjes")
#g.text(dt.Date.min(), dt.VirusGrowth_7d.min() * 0.9, "bron: corona dashboard update " + dateOfStats.strftime('%Y-%m-%d'), ha="left", fontsize=10, color='grey')
##plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0)

## Kaartjes

In [ ]:
annotated_map = map_data.copy().set_index('statcode').join(sewer_growth[sewer_growth.Date == sewer_growth.Date.max()].copy().set_index('VR'), how='left')

g = annotated_map.plot("VirusGrowth_7d", cmap=__BuRd(), vmin=-30, vmax=30, edgecolor="#cccccc", legend=True)
g.set_axis_off()
plt.tight_layout()

g.set_title("Groei riooldeeltjes", fontdict={'fontsize':18,'fontweight': 'bold'})
#g.set(xlabel="", ylabel="Riooldeeltjes")
#g.text(dt.Date.min(), dt.VirusGrowth_7d.min() * 0.9, "bron: corona dashboard update " + dateOfStats.strftime('%Y-%m-%d'), ha="left", fontsize=8, color='grey')

#g = sns.FacetGrid(annotated_map, col="Security_region_name", col_wrap=4, margin_titles=True, legend_out=True)
#def map_plot(data, **kws):
#    ax = plt.gca()
#    data.plot("VirusGrowth_7d", cmap=__BuRd(), ax= ax, vmin=-100, vmax=100, edgecolor="#cccccc", legend=True)
#    ax.set_axis_off()
#
#
#g = g.map_dataframe(map_plot)
#
#g.set_titles(col_template = '{col_name}')
#g.add_legend(loc='upper right')
#
#g.despine()
#plt.tight_layout()
#plt.subplots_adjust(hspace=0.1, wspace=0.05, top=0.93)
#g.fig.suptitle('Groei percentage deeltjes in riool (7d gemiddelde)', size=16.0, weight='bold')
#g.fig.text(1,0.03, "bron: RIVM update " + dateOfStats.strftime('%Y-%m-%d') + " & CBS statistiek", ha="right", fontsize=12, color='grey')

